### Imports

In [67]:
import requests
import os
import json
from dotenv import load_dotenv
from assistants_api_object_pretty_printer import pretty_print, pretty_print_thread

load_dotenv()


openai_api_key = os.getenv('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()


## Step 1: Create Assistant

Assistants should be created once and referenced all throught via the assistant_id

assistant_id = "asst_y3eBA6MHVryI4X81SeS1U0Ni"

In [68]:
from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
    instructions="You are a friendly assistant.",
    name="Assistant Beta",
    tools = [],
    model="gpt-3.5-turbo-1106"
)
pretty_print(assistant)

### Utils: View Details of Existing Assistant

In [69]:
assistant_beta = client.beta.assistants.retrieve("asst_y3eBA6MHVryI4X81SeS1U0Ni")
pretty_print(assistant_beta)

{
  "id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "created_at": "1699689431",
  "description": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "name": "Assistant Beta",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters": "{'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['city']}",
        "description": "Get the current traffic condition in a city."
      },
      "type": "functi

#### Set Assistant ID as constant

In [70]:
# to be set in env variables
ASSISTANT_ID = "asst_y3eBA6MHVryI4X81SeS1U0Ni"

assistant_id = ASSISTANT_ID

### Utils: Update assistant (for example if you want to add/remove tools)

In [71]:
assistant_beta = client.beta.assistants.update(
  assistant_id,
  instructions="You are a friendly assistant.",
  name="Assistant Beta",
  tools=[
        {
            "type": "code_interpreter"
        },
        {
            "type": "function",
            "function": {
                "name": "getCurrentWeather",
                "description": "Get the weather in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string", 
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string", 
                            "enum": ["c", "f"]
                        }
                    },
                    "required": ["location"]
                }
            },
        }
    ],
  model="gpt-3.5-turbo-1106",
  #file_ids=["file-abc123", "file-abc456"],
)

pretty_print(assistant_beta)


{
  "id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "created_at": "1699689431",
  "description": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "name": "Assistant Beta",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    },
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    }
  ]
}


# Step 2: Create Thread

TIP: Threads can be assigned to different assistants (.e.g. same assistant but one powered by GPT3.5, another by GPT4)

In [72]:
thread = client.beta.threads.create()
pretty_print(thread)

{
  "id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr",
  "created_at": "1699792936",
  "metadata": "{}",
  "object": "thread"
}


## Step 3: Create Message

In [73]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hows the weather in Metro Manila today?"
)
pretty_print(message)

{
  "id": "msg_LbLLWQCSLEc2caBjqEmr0jbb",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hows the weather in Metro Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792937",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}


### Utils: Display Existing Messages in Thread ID

In [74]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_LbLLWQCSLEc2caBjqEmr0jbb",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hows the weather in Metro Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792937",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}


## Step 4: Create Run

In [75]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id,
  instructions="always use getCurrentWeather "
)

## Step 5: Wait for Run to finish then View

Note status value can be: 
- queued 
- in_progress
- requires_action
- cancelling
- cancelled 
- failed
- completed
- expired

In [76]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

run = wait_on_run(run, thread)
pretty_print(run)


{
  "id": "run_pBD7A4TrEUCcM0AfTMiA0yYq",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1699792938",
  "expires_at": "1699793538",
  "failed_at": null,
  "file_ids": [],
  "instructions": "always use getCurrentWeather ",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_FS5StqzzgjFsks9ZtDafRFgK",
          "function": {
            "arguments": "{'location': 'Metro Manila', 'unit': 'c'}",
            "name": "getCurrentWeather"
          },
          "type": "function"
        }
      ]
    },
    "type": "submit_tool_outputs"
  },
  "started_at": "1699792938",
  "status": "requires_action",
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr",
  "tools": [
    {
      "type": "code_interpreter"
    },
    {
      "function": {
        "name": "getCurrentWeather",
  

# If status = requires_action: 

### Initialize tool_call as var 

It's possible that it will have several index/page of tools to use based on the nature of the query

This way, we can just iterate on this object by calling the index position


In [77]:
tool_call = run.required_action.submit_tool_outputs.tool_calls
pretty_print(tool_call)

[
  {
    "id": "call_FS5StqzzgjFsks9ZtDafRFgK",
    "function": {
      "arguments": "{'location': 'Metro Manila', 'unit': 'c'}",
      "name": "getCurrentWeather"
    },
    "type": "function"
  }
]



## Step 6: Update Run with Tool Output

In [78]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
    {
      "tool_call_id": tool_call[0].id,
      "output": "19C"
    }
  ]
)
pretty_print(run)

{
  "id": "run_pBD7A4TrEUCcM0AfTMiA0yYq",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1699792938",
  "expires_at": "1699793538",
  "failed_at": null,
  "file_ids": [],
  "instructions": "always use getCurrentWeather ",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": null,
  "started_at": "1699792938",
  "status": "queued",
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr",
  "tools": [
    {
      "type": "code_interpreter"
    },
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    }
  ]
}


### Display Content of Thread again

In [79]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_LbLLWQCSLEc2caBjqEmr0jbb",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hows the weather in Metro Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792937",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}


### Utils: View Run Steps

In [80]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(pretty_print(step_details), indent=4))

{
  "tool_calls": [
    {
      "id": "call_FS5StqzzgjFsks9ZtDafRFgK",
      "function": {
        "arguments": "{'location': 'Metro Manila', 'unit': 'c'}",
        "name": "getCurrentWeather",
        "output": "19C"
      },
      "type": "function"
    }
  ],
  "type": "tool_calls"
}
null
{
  "message_creation": {
    "message_id": "msg_A5raofTPapM5hQExs6LgMweX"
  },
  "type": "message_creation"
}
null


# Asking follow up questions

Current conversation inside the thread:

User: "Hows the weather in Metro Manila today?"

AI: "The weather in Metro Manila today is 19\u00b0C."

### Follow up question:

User: "Why is it so cold?

## Step 7: Update Thread with new message

In [81]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Why is it so cold?"
)
pretty_print(message)

{
  "id": "msg_YFCBU8KG71w8pXXkRBnS7qIA",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Why is it so cold?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792942",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}


### Utils: Display Existing Messages in Thread

In [82]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_YFCBU8KG71w8pXXkRBnS7qIA",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Why is it so cold?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792942",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}
{
  "id": "msg_A5raofTPapM5hQExs6LgMweX",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "The current temperature in Metro Manila is 19\u00b0C."
      },
      "type": "text"
    }
  ],
  "created_at": "1699792941",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_pBD7A4TrEUCcM0AfTMiA0yYq",
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}
{
  "id": "msg_LbLLWQCSLEc2caBjqEmr0jbb",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotat

## Step 8: Re-Execute Run

Removed the instructions param ("always use getCurrentWeather") so AI won't use a tool.

In [83]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id,  
)

## Step 9: Wait for Run to finish then View

In [84]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,            
        )
        time.sleep(0.5)
    return run

run = wait_on_run(run, thread)
pretty_print(run)

{
  "id": "run_Tl2NRVKxlxQYxPYtfephAd43",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "cancelled_at": null,
  "completed_at": "1699792944",
  "created_at": "1699792943",
  "expires_at": null,
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": null,
  "started_at": "1699792943",
  "status": "completed",
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr",
  "tools": [
    {
      "type": "code_interpreter"
    },
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    }
  ]
}


### Utils: Display Existing Messages in Thread 
to check the Message added by the assistant based on the last user input

In [85]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_trrQjwa4ohFvr5hc0YZClmBi",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "The temperature is relatively lower, possibly due to weather patterns, climate, or time of the year. If you'd like, I can provide more information about the weather and temperature trends in Metro Manila."
      },
      "type": "text"
    }
  ],
  "created_at": "1699792943",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_Tl2NRVKxlxQYxPYtfephAd43",
  "thread_id": "thread_AIl0jp0eeQdXY00zAi2Hjyxr"
}
{
  "id": "msg_YFCBU8KG71w8pXXkRBnS7qIA",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Why is it so cold?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792942",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thr

### Utils: Pretty Print Thread Content

In [86]:
print(pretty_print_thread(thread_messages))

User: "Hows the weather in Metro Manila today?"
Assistant: "The current temperature in Metro Manila is 19°C."
User: "Why is it so cold?"
Assistant: "The temperature is relatively lower, possibly due to weather patterns, climate, or time of the year. If you'd like, I can provide more information about the weather and temperature trends in Metro Manila."



# New Thread/Same Assistant - AI Uses Multiple Tools for a Single Query

## Step 1: Update Assisstant

Add new tool "getCurrentTraffic"

In [98]:
assistant_beta = client.beta.assistants.update(
  assistant_id,
  instructions="You are a friendly assistant.",
  name="Assistant Beta",
  tools=[        
        {
            "type": "function",
            "function": {
                "name": "getCurrentWeather",
                "description": "Get the weather in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string", 
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string", 
                            "enum": ["c", "f"]
                        }
                    },
                    "required": ["location"]
                }
            },
        },
        {
            "type": "function",
            "function": {
                "name": "getCurrentTraffic",
                "description": "Get the current traffic condition in a city.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The city and state e.g. San Francisco, CA"
                        }
                    },
                    "required": ["city"]
                }
            }
        }
        
    ],
  model="gpt-3.5-turbo-1106",
  #file_ids=["file-abc123", "file-abc456"],
)

pretty_print(assistant_beta)


{
  "id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "created_at": "1699689431",
  "description": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "name": "Assistant Beta",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters": "{'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['city']}",
        "description": "Get the current traffic condition in a city."
      },
      "type": "functi

## Step 2: Create New Thread

In [99]:
new_thread = client.beta.threads.create()
pretty_print(new_thread)

{
  "id": "thread_fxj26yBmoicBUHxlVu9RcPnv",
  "created_at": "1699792980",
  "metadata": "{}",
  "object": "thread"
}


## Step 3: Create New Message

new message should be answered using the 2 available tools

In [100]:
new_message = client.beta.threads.messages.create(
    thread_id=new_thread.id,
    role="user",
    content="Hows the weather and traffic condition in Metro Manila today?"
)
pretty_print(new_message)

{
  "id": "msg_MkaQimkDj70CAiYcD2F5eJtw",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hows the weather and traffic condition in Metro Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792994",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_fxj26yBmoicBUHxlVu9RcPnv"
}


### Utils: Display Existing Messages in Thread ID

In [101]:
new_thread_messages = client.beta.threads.messages.list(new_thread.id)

for index in range(len(new_thread_messages.data)):
    pretty_print(new_thread_messages.data[index])

{
  "id": "msg_MkaQimkDj70CAiYcD2F5eJtw",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hows the weather and traffic condition in Metro Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792994",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_fxj26yBmoicBUHxlVu9RcPnv"
}


## Step 4: Create Run

In [102]:
new_run = client.beta.threads.runs.create(
  thread_id=new_thread.id,
  assistant_id=assistant_id,
  # optional param if you want to override 'system message' on a per run basis.
  instructions="You are a friendly assistant with access to tools. Use all available tools if necessary to answer the user query.",
  #tools=tools < -- available param if you want to modify tool set on a per run basis.
)

## Step 5: Wait for Run to finish then View

In [109]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

new_run = wait_on_run(new_run, new_thread)
pretty_print(new_run)

{
  "id": "run_CVJMEJawgagaDCMEZUuH6Rqc",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1699793009",
  "expires_at": "1699793609",
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant with access to tools. Use all available tools if necessary to answer the user query.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_gwyyPb5IabNnYBQnXAoVO0Mk",
          "function": {
            "arguments": "{'location': 'Metro Manila, Philippines', 'unit': 'c'}",
            "name": "getCurrentWeather"
          },
          "type": "function"
        },
        {
          "id": "call_yUIffc5AzhIBPsXdiCgNmb8J",
          "function": {
            "arguments": "{'city': 'Metro Manila, Philippines'}",
            "name": "getCurrentTraffic"
  

# If status = requires_action:

### Initialize tool_call as var 

In [104]:
new_tool_call = new_run.required_action.submit_tool_outputs.tool_calls
pretty_print(new_tool_call)

[
  {
    "id": "call_gwyyPb5IabNnYBQnXAoVO0Mk",
    "function": {
      "arguments": "{'location': 'Metro Manila, Philippines', 'unit': 'c'}",
      "name": "getCurrentWeather"
    },
    "type": "function"
  },
  {
    "id": "call_yUIffc5AzhIBPsXdiCgNmb8J",
    "function": {
      "arguments": "{'city': 'Metro Manila, Philippines'}",
      "name": "getCurrentTraffic"
    },
    "type": "function"
  }
]


## Step 6: Update Run with Tool Output

In [105]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=new_thread.id,
  run_id=new_run.id,
  tool_outputs=[
    {
      "tool_call_id": new_tool_call[0].id,
      "output": "19C"
    },
    {
      "tool_call_id": new_tool_call[1].id,
      "output": "carmageddon levels per the usual"
    }

  ]
)
pretty_print(run)

{
  "id": "run_CVJMEJawgagaDCMEZUuH6Rqc",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1699793009",
  "expires_at": "1699793609",
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant with access to tools. Use all available tools if necessary to answer the user query.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": null,
  "started_at": "1699793009",
  "status": "queued",
  "thread_id": "thread_fxj26yBmoicBUHxlVu9RcPnv",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },

### Display Content of Thread again

In [106]:
new_thread_messages = client.beta.threads.messages.list(new_thread.id)

for index in range(len(new_thread_messages.data)):
    pretty_print(new_thread_messages.data[index])

{
  "id": "msg_J5gpI4FhDSlUYTFfbiOIWNLA",
  "assistant_id": "asst_y3eBA6MHVryI4X81SeS1U0Ni",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "The weather in Metro Manila today is 19\u00b0C. As for the traffic condition, it's at \"carmageddon levels per the usual.\" Stay safe out there!"
      },
      "type": "text"
    }
  ],
  "created_at": "1699793058",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_CVJMEJawgagaDCMEZUuH6Rqc",
  "thread_id": "thread_fxj26yBmoicBUHxlVu9RcPnv"
}
{
  "id": "msg_MkaQimkDj70CAiYcD2F5eJtw",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hows the weather and traffic condition in Metro Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1699792994",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_fxj26yBmoicBU

### Utils: Pretty Print Thread Content

In [107]:
print(pretty_print_thread(new_thread_messages))

User: "Hows the weather and traffic condition in Metro Manila today?"
Assistant: "The weather in Metro Manila today is 19°C. As for the traffic condition, it's at "carmageddon levels per the usual." Stay safe out there!"



### Utils: View Run Steps

In [108]:
new_run_steps = client.beta.threads.runs.steps.list(
    thread_id=new_thread.id, run_id=new_run.id, order="asc"
)

for step in new_run_steps.data:
    step_details = step.step_details
    print(json.dumps(pretty_print(step_details), indent=4))

{
  "tool_calls": [
    {
      "id": "call_gwyyPb5IabNnYBQnXAoVO0Mk",
      "function": {
        "arguments": "{'location': 'Metro Manila, Philippines', 'unit': 'c'}",
        "name": "getCurrentWeather",
        "output": "19C"
      },
      "type": "function"
    },
    {
      "id": "call_yUIffc5AzhIBPsXdiCgNmb8J",
      "function": {
        "arguments": "{'city': 'Metro Manila, Philippines'}",
        "name": "getCurrentTraffic",
        "output": "carmageddon levels per the usual"
      },
      "type": "function"
    }
  ],
  "type": "tool_calls"
}
null
{
  "message_creation": {
    "message_id": "msg_J5gpI4FhDSlUYTFfbiOIWNLA"
  },
  "type": "message_creation"
}
null
